<h1>Titanic - Machine Learning from Disaster</h1>

In [1]:
import numpy as np
import pandas as pd
import sklearn.tree as tree
from sklearn.tree import DecisionTreeClassifier
from sklearn import preprocessing

<h2>Visualización de los Datos</h2>

In [2]:
train_data  = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")

In [3]:
train_data.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


<h2>Interacción con los Datos</h2>

<p>Se calcula y compara el porcentaje de mujeres y hombres sobrevivientes.</p>

In [5]:
woman = (train_data[train_data["Sex"] == "female"])["Survived"]
rate_women = woman.mean()

print(f"Porcentaje de mujeres sobrevivientes: {rate_women*100:.2f} %")

Porcentaje de mujeres sobrevivientes: 74.20 %


In [6]:
men = (train_data[train_data["Sex"] == "male"])["Survived"]
rate_men = men.mean()

print(f"Porcentaje de hombres sobrevivientes: {rate_men*100:.2f} %")

Porcentaje de hombres sobrevivientes: 18.89 %


<h2>Pre-procesamiento de Datos</h2>

<h3>Limpieza de Datos</h3>

In [77]:
# Verifico si faltan datos
train_data.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [104]:
# Elimino las filas sin datos y selecciono las columnas apropiadas para el entrenamiento
trainset_data = train_data.dropna(subset=["Age", "Embarked"])[["Survived", "Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
print(trainset_data.isna().sum())
trainset_data.head()

Survived    0
Pclass      0
Sex         0
Age         0
SibSp       0
Parch       0
Fare        0
Embarked    0
dtype: int64


,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,22.0,1,0,7.2500,S
1,1,1,female,38.0,1,0,71.2833,C
2,1,3,female,26.0,0,0,7.9250,S
3,1,1,female,35.0,1,0,53.1000,S
4,0,3,male,35.0,0,0,8.0500,S


In [106]:
x_train = trainset_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]]
x_train = x_train.to_numpy()
x_train[0:5]

array([[3, 'male', 22.0, 1, 0, 7.25, 'S'],
       [1, 'female', 38.0, 1, 0, 71.2833, 'C'],
       [3, 'female', 26.0, 0, 0, 7.925, 'S'],
       [1, 'female', 35.0, 1, 0, 53.1, 'S'],
       [3, 'male', 35.0, 0, 0, 8.05, 'S']], dtype=object)

<h3>Codificación de valores categóricos</h3>

In [ ]:
le_sex = preprocessing.LabelEncoder()
le_sex.fit(["male", "female"])
x_train[:, 1] = le_sex.transform(x_train[:, 1])

le_embarked = preprocessing.LabelEncoder()
le_embarked.fit(["C", "Q", "S"])
x_train[:, 6] = le_embarked.transform(x_train[:, 6])

In [111]:
print(x_train.shape)
x_train[0:5]

(712, 7)


array([[3, 1, 22.0, 1, 0, 7.25, 2],
       [1, 0, 38.0, 1, 0, 71.2833, 0],
       [3, 0, 26.0, 0, 0, 7.925, 2],
       [1, 0, 35.0, 1, 0, 53.1, 2],
       [3, 1, 35.0, 0, 0, 8.05, 2]], dtype=object)

In [113]:
y_train = trainset_data["Survived"]
print(y_train.shape)
y_train[0:5]

(712,)


0    0
1    1
2    1
3    1
4    0
Name: Survived, dtype: int64

<h2>Entrenamiento usando Arbol de Desiciones</h2>

In [114]:
titanicTree = DecisionTreeClassifier(criterion='entropy', max_depth=4)
titanicTree

DecisionTreeClassifier(criterion='entropy', max_depth=4)

In [115]:
titanicTree.fit(x_train, y_train)

DecisionTreeClassifier(criterion='entropy', max_depth=4)

<h3>Predicciones</h3>

In [120]:
x_test = test_data[["Pclass", "Sex", "Age", "SibSp", "Parch", "Fare", "Embarked"]].to_numpy()

# Reemplazo valores categórigos por valores numéricos
x_test[:, 1] = le_sex.transform(x_test[:, 1])

x_test[:, 6] = le_embarked.transform(x_test[:, 6])

print(x_test.shape)
x_test

(418, 7)


array([[3, 1, 34.5, ..., 0, 7.8292, 1],
       [3, 0, 47.0, ..., 0, 7.0, 2],
       [2, 1, 62.0, ..., 0, 9.6875, 1],
       ...,
       [3, 1, 38.5, ..., 0, 7.25, 2],
       [3, 1, nan, ..., 0, 8.05, 2],
       [3, 1, nan, ..., 1, 22.3583, 0]], dtype=object)

In [121]:
predTree = titanicTree.predict(x_test)

In [123]:
print(predTree.shape)
predTree

(418,)


array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,

<h3>Exportar los resultados</h3>

In [ ]:
test_data["Survived"] = predTree

In [128]:
test_data[["PassengerId", "Survived"]].to_csv("titanicPredition", index=False)